In [108]:
import xarray as xr
import pandas as pd
import numpy as np
import pathlib
from pysheds.grid import Grid
import rasterio
import matplotlib.pyplot as plt


from bokeh.plotting import figure, column, show
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

In [109]:
# Define the path to the data folder
base_path = "C:/Users/vries_cy/OneDrive - Stichting Deltares/Indonesia case for students"
data_path = "semarang/hydromt_data_1/merit_hydro/"

data_folder = pathlib.Path(base_path, data_path)


In [117]:
# Retrieving and preparing the data
elevation = "elevtn.tif"
elevation_data = pathlib.Path(data_folder, elevation)
grid = Grid.from_raster(elevation_data.as_posix())
dem = np.flipud(grid.read_raster(elevation_data.as_posix()))
dem[dem < 0] = np.nan
dem_view = grid.view(dem, nodata=np.nan)


def wgs84_to_web_mercator(lat, lon):
    # Function to convert to mercator for Bokeh use
    k = 6378137
    x = lon * (k * np.pi/180.0)
    y = np.log(np.tan((90 + lat) * np.pi/360.0)) * k
    return x, y

TOOLTIPS = [("value", "@image")]

# Setting up the Bokeh figure with a background map
p = figure(x_axis_type="mercator", y_axis_type="mercator", tooltips=TOOLTIPS)
p.add_tile("CartoDB Positron", retina=True)

# Setting the extent in mercator coordinates
gridl1 = wgs84_to_web_mercator(grid.extent[2], grid.extent[0])
gridl2 = wgs84_to_web_mercator(grid.extent[3], grid.extent[1])

# Creating an image of the DEM data
source = ColumnDataSource(data=dict(image=[dem_view]))
dem_image = p.image(
    image='image',
    source=source,
    x=gridl1[0],
    y=gridl1[1],
    dw=gridl2[0] - gridl1[0],
    dh=gridl2[1] - gridl1[1],
    palette="Spectral11",
    alpha=0.5
)
# Create a color bar from the scatter glyph renderer
color_bar = dem_image.construct_color_bar(width=10)
p.add_layout(color_bar, 'right')

# Creating the slider to slide through the years
time_slider = Slider(title="Day in year", start=0, end=255, value=0, step=1)

# Adding callback code
callback = CustomJS(args=dict(source=source, val=time_slider),
                    code="""
    const data = source.data;
    const freq = val.value;
    const x = data['x'];
    const y = data['y'];
    for (var i = 0; i < x.length; i++) {
        y[i] = Math.sin(freq*x[i]);
    }
    source.change.emit();
""")
  
time_slider.js_on_change('value', callback)

show(column(p, time_slider))